# 서울시 구별 CCTV 현황 분석
- 서울시 CCTV 설치현황 데이터
- 서울시 구별 주민등록인구 통계 데이터

In [1]:
!apt-get install -y fonts-nanum > /dev/null
!fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib > /dev/null
# 런타임 다시 시작

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumBarunGothic')

## 1. CCTV 데이터 파악하고 정리하기

In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv to 서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv


In [3]:
cctv = pd.read_csv(filename, skiprows=1, encoding='euc-kr')
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289


In [4]:
# Null data 있는지 확인하기
cctv.isnull().sum().sum()

0

In [7]:
# 1000 단위 구분기호 없애고 정수로 변환하기
for column in cctv.columns[1:]:
    cctv[column] = cctv[column].apply(lambda x: int(x.replace(',','')))
cctv.head(3)

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289


In [8]:
cctv['2017년_이전'] = cctv.총계 - cctv['2018년'] - cctv['2019년'] - cctv['2020년'] - cctv['2021년']
cctv['최근증가율'] = ((cctv['2018년']+cctv['2019년']+cctv['2020년']) / cctv['2017년_이전'] * 100).round(2)
cctv.head(3)


,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2017년_이전,최근증가율
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608,42455,77.66
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43,1435,20.49
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289,1142,78.98


In [9]:
# 구분, 총계, 최근증가율 컬럼만 추출하고 컬럼명을 구별, CCTV댓수로 변경하기
cctv = cctv[['구분', '총계','최근증가율']]
cctv.rename(columns={'구분':'구별', '총계':'CCTV댓수'}, inplace=True)
cctv.head(3)

,구별,CCTV댓수,최근증가율
0,계,77032,77.66
1,종로구,1772,20.49
2,중 구,2333,78.98


In [10]:
# 구 이름에서 공백지우기
# cctv['구별'] = cctv.구별.apply(lambda x: x.replace(' ',''))
cctv['구별'] = cctv.구별.str.replace(' ','')
cctv.head()

,구별,CCTV댓수,최근증가율
0,계,77032,77.66
1,종로구,1772,20.49
2,중구,2333,78.98
3,용산구,2383,78.64
4,성동구,3602,46.13


In [11]:
# 첫번째 행 지우기
cctv.drop([0], inplace=True)

In [12]:
# 최근증가율 Top 5
cctv.sort_values(by='최근증가율', ascending=False).head()

,구별,CCTV댓수,최근증가율
9,강북구,2462,446.95
24,송파구,2854,203.19
7,중랑구,3296,191.07
18,금천구,2374,149.47
12,은평구,3791,117.86


In [13]:
# CCTV 댓수 Top 5
cctv.sort_values(by='CCTV댓수', ascending=False).head()

,구별,CCTV댓수,최근증가율
23,강남구,6502,61.64
21,관악구,4942,49.91
17,구로구,4075,48.83
8,성북구,3958,94.53
12,은평구,3791,117.86


## 2. 인구데이터 정리하기

In [18]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시주민등록인구(구별)통계.tsv.txt to 서울시주민등록인구(구별)통계.tsv.txt


In [19]:
pop = pd.read_csv(filename, sep='\t', skiprows=2)
pop.head(3)

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2021.2/4,합계,"4,405,638","9,795,426","4,756,178","5,039,248","9,565,990","4,649,442","4,916,548","229,436","106,736","122,700",2.17,"1,576,971"
1,2021.2/4,종로구,"73,525","155,106","75,009","80,097","146,029","70,985","75,044","9,077","4,024","5,053",1.99,"27,605"
2,2021.2/4,중구,"62,897","132,259","64,526","67,733","123,016","60,019","62,997","9,243","4,507","4,736",1.96,"23,980"


In [20]:
# Null data 있는지 확인하기
pop.isnull().sum().sum()

0

In [21]:
# '자치구','계','계.1','계.2','65세이상고령자‘ 컬럼만 추출하고 이름을 변경
pop = pop[['자치구','계','계.1','계.2','65세이상고령자']]
pop.columns = ['구별','인구수','내국인','외국인','고령자']
pop.head(3)

,구별,인구수,내국인,외국인,고령자
0,합계,"9,795,426","9,565,990","229,436","1,576,971"
1,종로구,"155,106","146,029","9,077","27,605"
2,중구,"132,259","123,016","9,243","23,980"


In [22]:
# 1000 단위 구분기호 없애고 정수로 변환하기
for column in pop.columns[1:]:
    pop[column] = pop[column].apply(lambda x: int(x.replace(',','')))

In [23]:
# 첫번째 행 지우기
pop.drop([0], inplace=True)
pop.head(3)

,구별,인구수,내국인,외국인,고령자
1,종로구,155106,146029,9077,27605
2,중구,132259,123016,9243,23980
3,용산구,240665,226378,14287,38884
